In [22]:
from IPython.display import Audio
import os
from pathlib import Path
i = 2
audio_dir = Path('/nas/longleaf/home/tis/wd/V2Music/preprocessing/data/audio')
audio_files = list(audio_dir.glob("*.m4a")) + list(audio_dir.glob("*.webm"))
len(audio_files)
#audios = os.listdir(audio_dir)
#audio_src = os.path.join(audio_dir, audios[i])
#Audio(audio_src)

19169

In [23]:
Audio("/work/users/t/i/tis/V2Music/preprocessing/data/audio/5DrfgXj-lCI.webm")

In [5]:
import os
import math

def get_filesize(file_path):
    try:
        size_bytes = os.path.getsize(file_path)
    except OSError as e:
        return f"Error: {e}"
    
    if size_bytes == 0:
        return "0 B"

    size_units = ("B", "KB", "MB", "GB", "TB", "PB")
    unit_index = int(math.floor(math.log(size_bytes, 1024)))
    power = math.pow(1024, unit_index)
    size = round(size_bytes / power, 2)
    
    return f"{size} {size_units[unit_index]}"

In [19]:
import subprocess
from pathlib import Path

def get_bitrate(file_path: Path) -> int:
    """
    Use ffprobe to get the overall bitrate (in bits per second) from the file's format metadata.
    Returns the bitrate as an integer, or -1 if not available.
    """
    ffprobe_cmd = [
        "ffprobe",
        "-v", "error",
        "-show_entries", "format=bit_rate",
        "-of", "default=noprint_wrappers=1:nokey=1",
        str(file_path)
    ]
    try:
        output = subprocess.check_output(ffprobe_cmd, stderr=subprocess.DEVNULL).strip()
        return int(output.decode("utf-8"))
    except subprocess.CalledProcessError:
        return -1

def get_sample_rate(file_path: Path) -> int:
    """
    Use ffprobe to get the sample rate (in Hz) from the first audio stream.
    Returns the sample rate as an integer, or -1 if not available.
    """
    ffprobe_cmd = [
        "ffprobe",
        "-v", "error",
        "-select_streams", "a:0",
        "-show_entries", "stream=sample_rate",
        "-of", "default=noprint_wrappers=1:nokey=1",
        str(file_path)
    ]
    try:
        output = subprocess.check_output(ffprobe_cmd, stderr=subprocess.DEVNULL).strip()
        return int(output.decode("utf-8"))
    except subprocess.CalledProcessError:
        return -1

def find_min_max_metadata(folder: Path):
    """
    Iterates over all files in the provided folder (assumed to be audio files)
    and tracks the minimum and maximum bitrate as well as the minimum and maximum
    sample rate, printing the respective file names and values.
    """
    min_bitrate = None
    max_bitrate = None
    min_bitrate_file = None
    max_bitrate_file = None
    
    min_sample_rate = None
    max_sample_rate = None
    min_sample_file = None
    max_sample_file = None
    
    for file in folder.iterdir():
        if file.is_file():
            bitrate = get_bitrate(file)
            sample_rate = get_sample_rate(file)
            # Process bitrate if valid (> 0)
            if bitrate > 0:
                if min_bitrate is None or bitrate < min_bitrate:
                    min_bitrate = bitrate
                    min_bitrate_file = file
                if max_bitrate is None or bitrate > max_bitrate:
                    max_bitrate = bitrate
                    max_bitrate_file = file
            # Process sample rate if valid (> 0)
            if sample_rate > 0:
                if min_sample_rate is None or sample_rate < min_sample_rate:
                    min_sample_rate = sample_rate
                    min_sample_file = file
                if max_sample_rate is None or sample_rate > max_sample_rate:
                    max_sample_rate = sample_rate
                    max_sample_file = file

    print("Bitrate (bps):")
    if min_bitrate is not None and max_bitrate is not None:
        print(f"  Minimum bitrate: {min_bitrate} bps (File: {min_bitrate_file})")
        print(f"  Maximum bitrate: {max_bitrate} bps (File: {max_bitrate_file})")
    else:
        print("  No valid bitrate data found.")
    
    print("\nSample Rate (Hz):")
    if min_sample_rate is not None and max_sample_rate is not None:
        print(f"  Minimum sample rate: {min_sample_rate} Hz (File: {min_sample_file})")
        print(f"  Maximum sample rate: {max_sample_rate} Hz (File: {max_sample_file})")
    else:
        print("  No valid sample rate data found.")

# Define the folder path (make sure this folder contains only audio files)
folder_path = Path("/nas/longleaf/home/tis/wd/V2Music/preprocessing/data/audio")

# Execute the metadata extraction and print the results.
find_min_max_metadata(folder_path)

Bitrate (bps):
  Minimum bitrate: 3766 bps (File: /nas/longleaf/home/tis/wd/V2Music/preprocessing/data/audio/6ZwjeJosWFw.webm)
  Maximum bitrate: 388360 bps (File: /nas/longleaf/home/tis/wd/V2Music/preprocessing/data/audio/geDoEeQJlH0.m4a)

Sample Rate (Hz):
  Minimum sample rate: 44100 Hz (File: /nas/longleaf/home/tis/wd/V2Music/preprocessing/data/audio/--7Nv4RYjd8.m4a)
  Maximum sample rate: 48000 Hz (File: /nas/longleaf/home/tis/wd/V2Music/preprocessing/data/audio/--foBg2jWzk.webm)


In [2]:
print(get_filesize(audio_src))
print(get_filesize("/nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/separated/mdx_extra/NvP8UQAQEdU/minus_vocals.webm"))

NameError: name 'get_filesize' is not defined

In [8]:
!nvidia-smi

Tue Apr 15 00:08:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40S                    Off |   00000000:B6:00.0 Off |                    0 |
| N/A   38C    P0             83W /  350W |    3577MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

import demucs.separate
out_path = '/nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/separated'
#demucs.separate.main(["--two-stems", "vocals", "--other-method", "minus", "-n", "mdx_extra","--filename", "{track}.{ext}", "-o", out_path, audio_src])
demucs.separate.main([
    "--two-stems", "vocals",
   # "--other-method", "minus",
    "-n", "mdx_extra",
    "--filename", "{track}_{stem}.{ext}",
    "-o", out_path,
    *map(str, audio_files[0:5])  # Convert each Path to a string
])

Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /work/users/t/i/tis/V2Music/preprocessing/test_data/separated/mdx_extra
Separating track /nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/audio/1eysYMU3h_I.webm


100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 411.42seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 393.27seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 358.11seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 363.47seconds/s]
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 8 (GCC)
  configuration: --prefix=/work/users/t/i/tis/ffmpeg/ffmpeg_build --enable-openssl --disable-shared --enable-static --extra-cflags=-I/work/users/t/i/tis/ffmpeg/openssl/include --extra-ldflags=-L/work/users/t/i/tis/ffmpeg/openssl/lib
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.101 / 61. 19.101
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100

Separating track /nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/audio/Hp01_AkITdg.webm


100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 367.64seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 377.62seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 353.98seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 390.95seconds/s]
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 8 (GCC)
  configuration: --prefix=/work/users/t/i/tis/ffmpeg/ffmpeg_build --enable-openssl --disable-shared --enable-static --extra-cflags=-I/work/users/t/i/tis/ffmpeg/openssl/include --extra-ldflags=-L/work/users/t/i/tis/ffmpeg/openssl/lib
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.101 / 61. 19.101
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100

Separating track /nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/audio/NvP8UQAQEdU.webm


100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 392.08seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 377.35seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 374.66seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 381.35seconds/s]
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 8 (GCC)
  configuration: --prefix=/work/users/t/i/tis/ffmpeg/ffmpeg_build --enable-openssl --disable-shared --enable-static --extra-cflags=-I/work/users/t/i/tis/ffmpeg/openssl/include --extra-ldflags=-L/work/users/t/i/tis/ffmpeg/openssl/lib
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.101 / 61. 19.101
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100

Separating track /nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/audio/Q847tRfANXU.webm


100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 383.53seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 376.85seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 375.37seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 383.06seconds/s]
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 8 (GCC)
  configuration: --prefix=/work/users/t/i/tis/ffmpeg/ffmpeg_build --enable-openssl --disable-shared --enable-static --extra-cflags=-I/work/users/t/i/tis/ffmpeg/openssl/include --extra-ldflags=-L/work/users/t/i/tis/ffmpeg/openssl/lib
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.101 / 61. 19.101
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100

Separating track /nas/longleaf/home/tis/wd/V2Music/preprocessing/test_data/audio/W-KTCowRGqw.webm


100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 390.81seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 379.08seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 382.25seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 84.0/84.0 [00:00<00:00, 385.70seconds/s]
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 8 (GCC)
  configuration: --prefix=/work/users/t/i/tis/ffmpeg/ffmpeg_build --enable-openssl --disable-shared --enable-static --extra-cflags=-I/work/users/t/i/tis/ffmpeg/openssl/include --extra-ldflags=-L/work/users/t/i/tis/ffmpeg/openssl/lib
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.101 / 61. 19.101
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100

In [6]:
demucs.separate.list_models()

{'single': {'0d19c1c6': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/0d19c1c6-0f06f20e.th',
  '5d2d6c55': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/5d2d6c55-db83574e.th',
  '7d865c68': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/7d865c68-3d5dd56b.th',
  '7ecf8ec1': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/7ecf8ec1-70f50cc9.th',
  'a1d90b5c': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/a1d90b5c-ae9d2452.th',
  'c511e2ab': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/c511e2ab-fe698775.th',
  'cfa93e08': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/cfa93e08-61801ae1.th',
  'e51eebcc': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/e51eebcc-c1b80bdd.th',
  '6b9c2ca1': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/6b9c2ca1-3fd82607.th',
  'b72baf4e': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/b72baf4e-8778635e.th',
  '42e558d4': 'https://dl.fbaipublicfiles.com/demucs/mdx_final/42e558d4-196e0e1b.th',
  '305bc58f': 'https://dl.fbaipublicfiles.co

In [4]:
import sys
print(sys.path)

['/work/users/t/i/tis/.cache/python/cpython-3.9.21-linux-x86_64-gnu/lib/python39.zip', '/work/users/t/i/tis/.cache/python/cpython-3.9.21-linux-x86_64-gnu/lib/python3.9', '/work/users/t/i/tis/.cache/python/cpython-3.9.21-linux-x86_64-gnu/lib/python3.9/lib-dynload', '', '/work/users/t/i/tis/demucs/.venv/lib/python3.9/site-packages']


In [4]:
import demucs.api
separator = demucs.api.Separator(model="mdx_extra")
origin, separated = separator.separate_audio_file(audio_src)
print(f"Origin: {origin}")
print(f"Separated: {separated}")    
for file, sources in separated:
    print(f"File: {file}, Sources: {sources}")
    for stem, source in sources.items():
        print(f"Stem: {stem}, Source: {source}")
        #demucs.api.save_audio(source, f"{stem}_{file}", samplerate=separator.samplerate)

Origin: tensor([[ 9.0949e-13,  9.0949e-13,  9.0949e-13,  ...,  7.5741e-02,
          7.6514e-02,  7.8899e-02],
        [ 9.0949e-13,  9.0949e-13,  9.0949e-13,  ...,  3.1223e-03,
         -3.5399e-03, -4.6720e-03]])
Separated: {'drums': tensor([[-0.0001, -0.0003,  0.0001,  ..., -0.0122, -0.0016, -0.0173],
        [-0.0001, -0.0003,  0.0001,  ..., -0.0141, -0.0068, -0.0159]]), 'bass': tensor([[-0.0001,  0.0001,  0.0002,  ...,  0.0015, -0.0018, -0.0028],
        [-0.0001,  0.0001,  0.0002,  ...,  0.0009, -0.0012,  0.0022]]), 'other': tensor([[-9.4694e-05,  6.3471e-05,  2.5396e-04,  ...,  7.7087e-02,
          7.2026e-02,  6.8448e-02],
        [-4.1497e-05,  9.8705e-05,  2.6506e-04,  ...,  1.5013e-02,
          1.0944e-02,  1.2071e-02]]), 'vocals': tensor([[ 2.1216e-05,  1.7958e-04,  3.6497e-04,  ...,  1.7859e-03,
          1.2953e-03,  1.0898e-03],
        [ 3.5653e-05,  1.8994e-04,  4.0092e-04,  ..., -3.1740e-04,
         -7.6794e-04, -7.4759e-04]])}


ValueError: too many values to unpack (expected 2)

In [ ]:
!spleeter separate -h

In [13]:
#!spleeter separate -o output/ audio_src
!spleeter separate -p spleeter:2stems audio_src

/bin/bash: spleeter: command not found


In [ ]:
Audio('output/audio_example/accompaniment.wav')

In [ ]:
Audio('output/audio_example/vocals.wav')

## Encodec

In [1]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor

# load the model + processor (for pre-processing the audio)
model = EncodecModel.from_pretrained("facebook/encodec_48khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_48khz")

config.json:   0%|          | 0.00/803 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.3M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
audio_dir = Path('/nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra')
audio_files = list(audio_dir.glob("*.wav"))
print(len(audio_files))

19169


In [22]:
!nvidia-smi

Tue Apr 15 10:31:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40                     Off |   00000000:E3:00.0 Off |                    0 |
| N/A   45C    P0             83W /  300W |   12201MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
import glob
import numpy as np
import soundfile as sf
import librosa
import torch
from transformers import EncodecModel, AutoProcessor

# Load the model and processor.
model = EncodecModel.from_pretrained("facebook/encodec_48khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_48khz")
target_sr = processor.sampling_rate  # Typically 48000 Hz

# Path to your directory containing the WAV files.
data_dir = "/nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra"
wav_files = glob.glob(os.path.join(data_dir, "*.wav"))
if not wav_files:
    print("No WAV files found in the directory:", data_dir)
    exit(1)

batch_size = 2

# Move model to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Process files in batches.
#for i in range(0, len(wav_files), batch_size):
for i in range(0, 4, batch_size):
    batch_files = wav_files[i:i+batch_size]
    audio_list = []
    file_names = []
    
    # Preprocess each audio file in the current batch.
    for wav_file in batch_files:
        print("Reading file:", wav_file)
        audio, sr = sf.read(wav_file)
        
        # ----- Ensure stereo audio with correct shape (2, num_samples) -----
        if audio.ndim == 1:
            audio = np.vstack((audio, audio))
        elif audio.ndim == 2:
            if audio.shape[1] == 2:
                audio = audio.T
            elif audio.shape[1] > 2:
                audio = audio[:, :2].T
            else:
                audio = np.vstack((audio[:, 0], audio[:, 0]))
        else:
            raise ValueError("Unsupported audio dimensions for file: " + wav_file)
        
        # ----- Resample if needed -----
        if sr != target_sr:
            resampled_channels = []
            for ch in range(audio.shape[0]):
                resampled = librosa.resample(audio[ch], orig_sr=sr, target_sr=target_sr)
                resampled_channels.append(resampled)
            audio = np.vstack(resampled_channels)
        
        audio_list.append(audio)
        file_names.append(os.path.basename(wav_file))
        print(audio_list[0].shape)
    
    # Batch Processing with the Processor and Model.
    inputs = processor(
        raw_audio=audio_list,
        sampling_rate=target_sr,
        padding=True,
        return_tensors="pt",
    )
    
    # Move inputs to the same device as the model.
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
        audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]
        #outputs = model(inputs["input_values"], padding_mask=inputs["padding_mask"])
        #audio_values = outputs.audio_values

    audio_values = audio_values.cpu().numpy()

    output_dir = os.path.join(data_dir, "encodec")
    os.makedirs(output_dir, exist_ok=True)

    # Compute the valid lengths (per sample) using the padding mask.
    valid_lengths = inputs["padding_mask"].sum(dim=1)
    for j, fname in enumerate(file_names):
        valid_len = int(valid_lengths[j].item())
        # Crop the output to its valid length (channels, valid_length).
        output = audio_values[j, :, :valid_len]
        # Transpose output to (valid_length, channels) for writing.
        output = output.T
        output_filename = os.path.join(output_dir, fname)
        # Uncomment the line below to save the file.
        sf.write(output_filename, output, target_sr)
        print("Saved processed audio to:", output_filename)

Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--7Nv4RYjd8_no_vocals.wav
(2, 1440009)
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--foBg2jWzk_no_vocals.wav
(2, 1440009)
Saved processed audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/encodec/--7Nv4RYjd8_no_vocals.wav
Saved processed audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/encodec/--foBg2jWzk_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--gdB-nnQkU_no_vocals.wav
(2, 1439689)
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--iC9GzoZEU_no_vocals.wav
(2, 1439689)
Saved processed audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/encodec/--gdB-nnQkU_no_vocals.wav
Saved processed audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/encodec/--iC9GzoZEU_no_vocals.wav


In [12]:
import os
import glob
import numpy as np
import soundfile as sf
import librosa
import torch
from transformers import EncodecModel, AutoProcessor
from torch.amp import autocast


# Load the model and processor.
model = EncodecModel.from_pretrained("facebook/encodec_48khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_48khz")
target_sr = processor.sampling_rate  # Typically 48000 Hz

# Path to your directory containing the WAV files.
data_dir = "/nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra"
wav_files = glob.glob(os.path.join(data_dir, "*.wav"))
if not wav_files:
    print("No WAV files found in the directory:", data_dir)
    exit(1)

batch_size = 20

# Move model to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Directory to save the encoded torch tensors.
output_enc_dir = os.path.join(data_dir, "encoded_tensors")
os.makedirs(output_enc_dir, exist_ok=True)

# Encode files in batches
for i in range(0, 40, batch_size):
    batch_files = wav_files[i:i+batch_size]
    audio_list = []
    # Preprocess each audio file in the current batch (correct sampling rate + stereo)
    for wav_file in batch_files:
        print("Reading file:", wav_file)
        audio, sr = sf.read(wav_file)
        
        # ----- Ensure stereo audio with correct shape (2, num_samples) -----
        if audio.ndim == 1:
            audio = np.vstack((audio, audio))
        elif audio.ndim == 2:
            if audio.shape[1] == 2:
                audio = audio.T
            elif audio.shape[1] > 2:
                audio = audio[:, :2].T
            else:
                audio = np.vstack((audio[:, 0], audio[:, 0]))
        else:
            raise ValueError("Unsupported audio dimensions for file: " + wav_file)
        
        # ----- Resample if needed -----
        if sr != target_sr:
            resampled_channels = []
            for ch in range(audio.shape[0]):
                resampled = librosa.resample(audio[ch], orig_sr=sr, target_sr=target_sr)
                resampled_channels.append(resampled)
            audio = np.vstack(resampled_channels)
        
        audio_list.append(audio)
       
    
    # Batch Processing with the Processor and Model.
    inputs = processor(
        raw_audio=audio_list,
        sampling_rate=target_sr,
        padding=True,
        return_tensors="pt",
    )
    
    # Move inputs to the same device as the model.
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        #outputs List[frame_number,(audio_codes, audio_scales)]
        #audio_codes with dimension (batch_size, channels, sequence_length)
       # with autocast(device_type='cuda'):
        encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"], bandwidth=24.0)

    audio_codes = encoder_outputs.audio_codes.transpose(0,1).cpu()
    audio_scales = torch.stack(encoder_outputs.audio_scales, dim=1).cpu()

    #shape padding_mask: batch size x samples 
    # For each file in the batch, save a dictionary with the encoded tensors.
    file_format = "pt"  # Change this to "pt" or "npz" as needed
    # Save the encoded tensors for each file in the batch.
    for j, wav_file in enumerate(batch_files):
        output_file = os.path.join(output_enc_dir, f"{os.path.splitext(os.path.basename(wav_file))[0]}.{file_format}")
        
        if file_format == "pt":
            encoded_dict = {
                "audio_codes": audio_codes[j],
                "audio_scales": audio_scales[j],
                "padding_mask": inputs["padding_mask"][j]
            }
            torch.save(encoded_dict, output_file)
        elif file_format == "npz":
            # Convert each tensor to NumPy arrays and save them in a compressed .npz file
            np.savez_compressed(
                output_file,
                audio_codes=audio_codes[j].numpy(),
                audio_scales=audio_scales[j].numpy(),
                padding_mask=inputs["padding_mask"][j].cpu().numpy()
            )
        else:
            raise ValueError("Unsupported file format: " + file_format)
        print(f"Saved encoded tensor to {output_file}")

Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--7Nv4RYjd8_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--foBg2jWzk_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--gdB-nnQkU_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--iC9GzoZEU_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--l0YXLEZqY_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--pPpFS5784_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/--v0dtBsvWM_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/-08JHVqBfOo_no_vocals.wav
Reading file: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/-0EL1b9ZUxw_no_vocals.wav
Reading file: /nas/longleaf/home/tis/

## Decode and Load

In [ ]:
def load_file(file_path, device):
        # Determine if the file is an NPZ file or a Torch file.
        if file_path.endswith('.npz'):
            # Load NPZ file with NumPy.
            npz_data = np.load(file_path)
            return {
                "audio_codes": torch.from_numpy(npz_data["audio_codes"]).to(device),
                "audio_scales": torch.from_numpy(npz_data["audio_scales"]).to(device),
                "padding_mask": torch.from_numpy(npz_data["padding_mask"]).to(device)
            }
        elif file_path.endswith('.pt'):
            return torch.load(file_path, map_location=device)
        else:
            raise ValueError(f"Unsupported file format: {file_path}. Supported formats are .npz and .pt.")

#Support .npz and .pt files
def load_audio_features(file_paths, device='cuda'):
    if not file_paths:
        raise ValueError("The file_paths list is empty.")
    
    B = len(file_paths)

    # Load the first file to determine shapes and data types.
    first_data = load_file(file_paths[0], device)
    codes_tensor = torch.empty((B,) + first_data["audio_codes"].shape,
                               dtype=first_data["audio_codes"].dtype, device=device)
    scales_tensor = torch.empty((B,) + first_data["audio_scales"].shape,
                                dtype=first_data["audio_scales"].dtype, device=device)
    mask_tensor = torch.empty((B,) + first_data["padding_mask"].shape,
                              dtype=first_data["padding_mask"].dtype, device=device)

    # Populate the first slot.
    codes_tensor[0] = first_data["audio_codes"]
    scales_tensor[0] = first_data["audio_scales"]
    mask_tensor[0]  = first_data["padding_mask"]

    print(codes_tensor[0].shape)
    print(scales_tensor[0].shape)
    print(mask_tensor[0].shape)
    # Iterate over the remaining file paths.
    for i in range(1, B):
        data = load_file(file_paths[i], device)

        current_codes = data["audio_codes"]
        current_scales = data["audio_scales"]
        current_mask   = data["padding_mask"]
        
        print(current_codes.shape)
        print(current_scales.shape)
        print(current_mask.shape)
        # Check for dimension consistency with the first element.
        if (current_codes.shape != codes_tensor[0].shape or
            current_scales.shape != scales_tensor[0].shape or
            current_mask.shape  != mask_tensor[0].shape):
            raise ValueError(f"Dimension mismatch in file {file_paths[i]}: "
                             f"Expected shapes {(codes_tensor[0].shape, scales_tensor[0].shape, mask_tensor[0].shape)} but got "
                             f"{(current_codes.shape, current_scales.shape, current_mask.shape)}")
        
        # Store the tensors.
        codes_tensor[i] = current_codes
        scales_tensor[i] = current_scales
        mask_tensor[i]   = current_mask

    # Reverse dimensions to move batch dimension to second place.
    return codes_tensor.transpose(0, 1), scales_tensor.transpose(0, 1), mask_tensor

In [ ]:
import os
import glob
import torch
import soundfile as sf
import numpy as np
from transformers import EncodecModel, AutoProcessor

def load_file(file_path, device):
        # Determine if the file is an NPZ file or a Torch file.
        if file_path.endswith('.npz'):
            # Load NPZ file with NumPy.
            npz_data = np.load(file_path)
            return {
                "audio_codes": torch.from_numpy(npz_data["audio_codes"]).to(device),
                "audio_scales": torch.from_numpy(npz_data["audio_scales"]).to(device),
                "padding_mask": torch.from_numpy(npz_data["padding_mask"]).to(device)
            }
        elif file_path.endswith('.pt'):
            return torch.load(file_path, map_location=device)
        else:
            raise ValueError(f"Unsupported file format: {file_path}. Supported formats are .npz and .pt.")

#Support .npz and .pt files
def load_audio_features(file_paths, device='cuda'):
    if not file_paths:
        raise ValueError("The file_paths list is empty.")
    
    B = len(file_paths)

    # Load the first file to determine shapes and data types.
    first_data = load_file(file_paths[0], device)
    codes_tensor = torch.empty((B,) + first_data["audio_codes"].shape,
                               dtype=first_data["audio_codes"].dtype, device=device)
    scales_tensor = torch.empty((B,) + first_data["audio_scales"].shape,
                                dtype=first_data["audio_scales"].dtype, device=device)
    mask_tensor = torch.empty((B,) + first_data["padding_mask"].shape,
                              dtype=first_data["padding_mask"].dtype, device=device)

    # Populate the first slot.
    codes_tensor[0] = first_data["audio_codes"]
    scales_tensor[0] = first_data["audio_scales"]
    mask_tensor[0]  = first_data["padding_mask"]

    print(codes_tensor[0].shape)
    print(scales_tensor[0].shape)
    print(mask_tensor[0].shape)
    # Iterate over the remaining file paths.
    for i in range(1, B):
        data = load_file(file_paths[i], device)

        current_codes = data["audio_codes"]
        current_scales = data["audio_scales"]
        current_mask   = data["padding_mask"]
        
        print(current_codes.shape)
        print(current_scales.shape)
        print(current_mask.shape)
        # Check for dimension consistency with the first element.
        if (current_codes.shape != codes_tensor[0].shape or
            current_scales.shape != scales_tensor[0].shape or
            current_mask.shape  != mask_tensor[0].shape):
            raise ValueError(f"Dimension mismatch in file {file_paths[i]}: "
                             f"Expected shapes {(codes_tensor[0].shape, scales_tensor[0].shape, mask_tensor[0].shape)} but got "
                             f"{(current_codes.shape, current_scales.shape, current_mask.shape)}")
        
        # Store the tensors.
        codes_tensor[i] = current_codes
        scales_tensor[i] = current_scales
        mask_tensor[i]   = current_mask

    # Reverse dimensions to move batch dimension to second place.
    return codes_tensor.transpose(0, 1), scales_tensor.transpose(0, 1), mask_tensor

# Load the model and processor.
model = EncodecModel.from_pretrained("facebook/encodec_48khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_48khz")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

target_sr = processor.sampling_rate  

# Set up directories.
data_dir = "/nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra"
encoded_dir = os.path.join(data_dir, "encoded_tensors")
decoded_dir = os.path.join(data_dir, "decoded_wav")
os.makedirs(decoded_dir, exist_ok=True)


file_format = "npz" 
encoded_files = glob.glob(os.path.join(encoded_dir, f"*.{file_format}"))
if not encoded_files:
    print("No encoded tensor files found in:", encoded_dir)
    exit(1)

batch_size = 2

# Process the encoded_files list in batches by slicing.
#for start_idx in range(0, len(encoded_files), batch_size):
for start_idx in range(0, 4, batch_size):
    
    batch_file_paths = encoded_files[start_idx:start_idx + batch_size]
    # Load the batched features using the helper function.
    batch_audio_codes, batch_audio_scales, batch_padding_mask = load_audio_features(batch_file_paths, device=device)
    with torch.no_grad():
        # Decode the batch of audio codes
        decoded_batch = model.decode(batch_audio_codes, batch_audio_scales, batch_padding_mask)[0]

    # Move the decoded audio to CPU and convert to numpy.
    decoded_batch = decoded_batch.cpu().numpy()
    # Compute the valid lengths for each file in the batch using the padding masks.
    valid_lengths = batch_padding_mask.sum(dim=1).cpu()

    # Process and save each sample in the current batch.
    for i, file in enumerate(batch_file_paths):
        valid_len = valid_lengths[i]
        # Assume decoded audio shape is (channels, time).
        sample_decoded = decoded_batch[i]
        # Crop the decoded audio to include only valid time steps.
        cropped_audio = sample_decoded[:, :valid_len]
        # Transpose from (channels, time) to (time, channels) for saving with soundfile.
        output_audio = cropped_audio.T
        output_file = os.path.join(decoded_dir, f"{os.path.splitext(os.path.basename(file))[0]}.wav")
        sf.write(output_file, output_audio, target_sr)
        print(f"Saved decoded audio to: {output_file}")

torch.Size([31, 16, 150])
torch.Size([31, 1, 1])
torch.Size([1473600])
torch.Size([31, 16, 150])
torch.Size([31, 1, 1])
torch.Size([1473600])
Saved decoded audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/decoded_wav/--7Nv4RYjd8_no_vocals.wav
Saved decoded audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/decoded_wav/--foBg2jWzk_no_vocals.wav
torch.Size([31, 16, 150])
torch.Size([31, 1, 1])
torch.Size([1473600])
torch.Size([31, 16, 150])
torch.Size([31, 1, 1])
torch.Size([1473600])
Saved decoded audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/decoded_wav/--gdB-nnQkU_no_vocals.wav
Saved decoded audio to: /nas/longleaf/home/tis/wd/V2Music/preprocessing/bg_audio/mdx_extra/decoded_wav/--iC9GzoZEU_no_vocals.wav


In [21]:
import glob
import os

folder = '/work/users/t/i/tis/V2Music/preprocessing/bg_audio/mdx_extra'
wav_files = glob.glob(os.path.join(folder, '*.wav'))
count = len(wav_files)
print(count)

19169
